In [38]:
#Import Libraries
import sys, os
import psycopg2
from psycopg2 import Error
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
import psycopg2.extras as extras
from IPython.display import clear_output
import numpy as np
import pandas as pd

In [2]:
# set up connection variables
db_host = "localhost"
db_port = "5432"
db_user = "postgres"
db_pass = "*********" #Input Password
db_name = "postgres"

# function to connect with postgres
def connect_postgres(db_host, db_port, db_user, db_pass, db_name):
    try:
        # Connect to an existing database
        connection = psycopg2.connect(host = db_host,
                                      port = db_port,
                                      user = db_user,
                                      password = db_pass,
                                      database = db_name)
        # Set auto-commit
        connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT);
        # Create a cursor to perform database operations
        cur = connection.cursor()
        # Print PostgreSQL details
        print("PostgreSQL server information")
        print(connection.get_dsn_parameters(), "\n")
        # Executing a SQL query
        cur.execute("SELECT version();")
        # Fetch result
        record = cur.fetchone()
        print("You are connected to - ", record, "\n")

    except (Exception, Error) as error:
        print("Error while connecting to PostgreSQL", error)
    else:
        return cur

In [3]:
# connect to postgres
cur = connect_postgres(db_host, db_port, db_user, db_pass, db_name)

PostgreSQL server information
{'user': 'postgres', 'channel_binding': 'prefer', 'dbname': 'postgres', 'host': 'localhost', 'port': '5432', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'sslsni': '1', 'ssl_min_protocol_version': 'TLSv1.2', 'gssencmode': 'disable', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 14.5, compiled by Visual C++ build 1914, 64-bit',) 



In [4]:
db_name = "stib_network"
cur.execute (
    F""" DROP DATABASE IF EXISTS {db_name} WITH (FORCE);
    """
)
print("SQL Status Output:\n", cur.statusmessage)

SQL Status Output:
 DROP DATABASE


In [5]:
# create db

cur.execute(
    f"""

    CREATE DATABASE {db_name}
        WITH
        OWNER = postgres
        TEMPLATE = template0
        ENCODING = 'UTF8'
        CONNECTION LIMIT = -1
        IS_TEMPLATE = False;
        
    """
)
print("SQL Status Output:\n", cur.statusmessage)

SQL Status Output:
 CREATE DATABASE


In [6]:
# connect to new database
cur = connect_postgres(db_host, db_port, db_user, db_pass, db_name)

PostgreSQL server information
{'user': 'postgres', 'channel_binding': 'prefer', 'dbname': 'stib_network', 'host': 'localhost', 'port': '5432', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'sslsni': '1', 'ssl_min_protocol_version': 'TLSv1.2', 'gssencmode': 'disable', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 14.5, compiled by Visual C++ build 1914, 64-bit',) 



In [59]:
## ALTER DATABASE OWNER

cur.execute (F""" 
ALTER DATABASE stib_network OWNER TO {db_user};""")

print("SQL Status Output: \n", cur.statusmessage)

SQL Status Output: 
 ALTER DATABASE


In [8]:
# SET client encoding

cur.execute("SET statement_timeout = 0; \
             SET lock_timeout = 0; \
             SET idle_in_transaction_session_timeout = 0; \
             SET client_encoding = 'UTF8'; \
             SET standard_conforming_strings = on; \
             SELECT pg_catalog.set_config('search_path', '', false); \
             SET check_function_bodies = false; \
             SET client_min_messages = warning; \
             SET row_security = off;"
)

print("SQL Status Output: \n", cur.statusmessage)

SQL Status Output: 
 SET


In [56]:
#creating relation
db_table = "vehicle_position"

# Create Schema
cur.execute (f"""
DROP TABLE IF EXISTS public.{db_table};
CREATE TABLE IF NOT EXISTS public.{db_table}
(Timestamp varchar(25) NOT NULL,
LineId varchar(15),
DirectionId varchar(15),
DistanceFromPoint varchar(15),
PointId varchar(15),
Date date NULL,
Time varchar(15) NULL,
Weekday INT NULL);""")

print("SQL Status Output: \n", cur.statusmessage)

In [52]:
# Read New Vehicle Position CSV
vehicle_positions_time = pd.read_csv('vehicle_positions_time.csv')
vehicle_positions_time.head(5)

,Timestamp,LineId,DirectionId,DistanceFromPoint,PointId,Date,Time,Weekday
0,1630914886924,1,8161,1,8012,2021-09-06,07:54:46,0
1,1630914886924,1,8162,0,8142,2021-09-06,07:54:46,0
2,1630914886924,1,8162,0,8282,2021-09-06,07:54:46,0
3,1630914886924,1,8731,0,8111,2021-09-06,07:54:46,0
4,1630914886924,1,8162,1,8062,2021-09-06,07:54:46,0


In [57]:
# function to insert the data from dataframe to db table for each case

def insert_df_into_db_table(df, table_name):
  
    tuples = [tuple(x) for x in df.to_numpy()]
    cols = ','.join(list(df.columns))
    query = "INSERT INTO %s(%s) VALUES %%s" % (table_name, cols)
    try:
        extras.execute_values(cur, query, tuples)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Insert Error: %s" % error)
    else:
        print(f'DB table {table_name} has been populated')

In [58]:
# apply function to insert the data from dataframe to db table for each case

insert_df_into_db_table(vehicle_positions_time, 'public.vehicle_position')

DB table public.vehicle_position has been populated


## END